In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import glob,os
import sys
sys.path.append("/Users/damoncrockett/ivpy/src/")
from ivpy import *
from PIL import Image

In [2]:
db = pd.read_pickle("/Users/damoncrockett/lmlproc/db.pkl")

In [3]:
db = db.loc[db.forma!="Package only"]
db = db.reset_index(drop=True)

In [4]:
fulldb = pd.read_csv("/Users/damoncrockett/Dropbox/lml/export_07_31_24.csv", encoding='latin-1')
fulldb = fulldb.fillna('')
fulldb['catalog'] = fulldb['Catalog Number'].apply(str) + fulldb['Secondary Catalog Number']
omits = list(fulldb.catalog.loc[fulldb.Omit_from_Paperbase==1]) + ['952']

In [5]:
#fulldb.loc[fulldb.catalog=='4148'].iloc[0]

In [6]:
# ADD
# all measurement trials
# legacy measurements?
# binning
# GEP
# full hex colors
# expressiveness
# steerable pyramid / t-SNE coords?
# UV (auc and something binary, probably)

In [7]:
# REMOVE
# flagged measurements
# there may be a lot of these for color

# thickness

In [8]:
tf = pd.read_pickle("/Users/damoncrockett/lmlproc/proc/thickness/raw_dedupe.pkl")

In [9]:
def median_of_lists(values):
    combined_list = sum(values, [])
    return pd.Series(combined_list).median()

In [10]:
tfm = tf.groupby('catalog')['thickness'].agg(median_of_lists).reset_index()

# gloss

In [11]:
import numpy as np

In [12]:
gf = pd.read_pickle("/Users/damoncrockett/lmlproc/proc/gloss/raw_dedupe.pkl")
gfp = pd.read_pickle("/Users/damoncrockett/lmlproc/proc/gloss/raw_dedupe_paul.pkl")

In [13]:
gf = gf[['cat','g60']]

In [14]:
gfp = gfp[['cat','g60']]

In [15]:
tmp = pd.concat([gf,gfp])

In [16]:
gfm = tmp.groupby('cat').agg(np.median).reset_index()

In [17]:
gfm.rename(columns={"cat":"catalog"},inplace=True)

# texture

In [18]:
xf = pd.read_pickle("/Users/damoncrockett/lmlproc/proc/texture/raw_corrections_dedupe.pkl")

In [19]:
medians = xf.groupby('catalog')['roughness'].median().rename('median_roughness')

In [20]:
xf = xf.merge(medians, on='catalog')

In [21]:
xf['diff'] = (xf['roughness'] - xf['median_roughness']).abs()

In [22]:
xfm = xf.sort_values(by='diff').drop_duplicates('catalog', keep='first')

In [23]:
xfm = xfm.drop(columns=['median_roughness', 'diff'])

In [24]:
xfm.rename(columns={"manbran":"manbran_x"},inplace=True)

In [25]:
xfm = xfm[['catalog','manbran_x','roughness','localpath']]

In [26]:
xfm = xfm.loc[~xfm.catalog.isin(omits)]

In [27]:
xfm = xfm.loc[xfm.catalog!='2047']

In [28]:
xfm = xfm.reset_index(drop=True)

In [29]:
len(xfm)

5957

In [30]:
xfm.roughness.loc[xfm.catalog=='2073z'] = None

In [31]:
import shutil

In [32]:
# for fpath in xfm.localpath:
#     shutil.copy(fpath,"/Users/damoncrockett/paperbase/src/assets/tifs/")
# note that this creates a vulnerability because below, when two items are the same distance from the median, 
# there is some randomness in which gets selected. Below is sort of a rewritten history of what I did.

In [33]:
# xfm['pbpath'] = ["/Users/damoncrockett/paperbase/src/assets/img/texture/" + os.path.basename(item) for item in xfm.localpath]

In [34]:
# xfm['pbpath'] = [os.path.splitext(item)[0] + ".jpg" for item in xfm.pbpath]

In [35]:
# for i in xfm.index:
#     pbpath = xfm.pbpath.loc[i]
#     if os.path.isfile(pbpath):
#         catalog = xfm.catalog.loc[i]
#         newpath = f'/Users/damoncrockett/paperbase/src/assets/img/texture/{catalog}.jpg'
#         os.rename(pbpath,newpath)

In [36]:
xfm['pbpath_new'] = [f'/Users/damoncrockett/paperbase/src/assets/img/texture/{c}.jpg' for c in xfm.catalog]

In [37]:
# actualfiles = glob.glob(os.path.join("/Users/damoncrockett/paperbase/src/assets/img/texture/","*.jpg"))

In [38]:
# for i in xfm.index:
#     p = xfm.pbpath_new.loc[i]
#     c = xfm.catalog.loc[i]
#     if not os.path.isfile(p):
#         pbpath = xfm.pbpath.loc[i]
#         fstem = os.path.basename(pbpath)[:-4]
#         cat_ = fstem.split("_")[0] + "_"
#         matches = [item for item in actualfiles if cat_ in item]
#         assert len(matches)==1
#         match = matches[0]
#         os.rename(match,p)

In [39]:
# for p in xfm.pbpath_new:
#     try:
#         Image.open(p)
#     except:
#         print(p)

In [40]:
#xfm.to_pickle('/Users/damoncrockett/Desktop/paperbase_OLD/xfm.pkl')

In [41]:
# attach(xfm,'pbpath_new')
# montage(thumb=16)

In [42]:
#db.loc[(db.catalog.str.startswith('17'))&(db.catalog.str.len()==3)&(db.man=='Agfa Gevaert')]

In [43]:
xfm = xfm[['catalog','roughness','pbpath_new']]

In [44]:
attach(xfm,'pbpath_new')

In [45]:
#show(xcol='roughness',xdomain=(0.05,1),thumb=128,notecol='catalog')

In [46]:
badcats = [
    '2401','2523','4870a','4870c','4870b',
    '1878','4142','1860e','4811','5407r',
    '3122','5407s','877p','1405'
]

In [47]:
# show(xfm.pbpath_new.loc[xfm.catalog.isin(badcats)],
#      thumb=200,
#      notecol=xfm.catalog.loc[xfm.catalog.isin(badcats)]).save('spurious_texture.jpg')

In [48]:
xfm.roughness.loc[xfm.catalog.isin(badcats)] = None

In [49]:
# for p in xfm.pbpath_new:
#     try:
#         Image.open(p)
#     except:
#         print(p)

# color

In [50]:
cf = pd.read_pickle("/Users/damoncrockett/lmlproc/proc/color/raw_omissions_corrections-mmode_deduped_catalog.pkl")

In [51]:
idxs = cf.loc[~cf.mmode.isin(['M0','M1','M2'])].index

In [52]:
cf.catalog.loc[idxs] = [item.split('_')[0] for item in cf.mmode.loc[idxs]]

In [53]:
cf.manbran.loc[idxs] = [item.split('_')[1].split('-')[0] for item in cf.mmode.loc[idxs]]
cf.mtrial.loc[idxs] = [item.split('_')[1].split('-')[1] for item in cf.mmode.loc[idxs]]

In [54]:
cf.mmode.loc[idxs] = "M0"

In [55]:
del cf['sample_type']
del cf['additional']

In [56]:
cf = cf.loc[cf.mmode=='M0']

In [57]:
cf = cf[[item for item in cf.columns if 'nm' not in item]]

In [58]:
for c in cf.catalog.unique():
    tmp = cf.loc[cf.catalog==c]
    manbrans = set(tmp.manbran)
    if len(manbrans) > 1:
        print(c)

5150


In [59]:
cf.manbran.loc[cf.catalog=='5150'] = 'cs'

#### medians

In [60]:
cf['catloc'] = [f'{cf.catalog.loc[i]}_{cf.mloc.loc[i]}' for i in cf.index]

In [61]:
# Group by 'catloc' and calculate the median of 'LAB_B' for each group
medians = cf.groupby('catloc')['LAB_B'].median().rename('median_LAB_B')

In [62]:
# Merge the median back to the original DataFrame
cf = cf.merge(medians, on='catloc')

In [63]:
# Calculate the absolute difference from the median
cf['diff'] = (cf['LAB_B'] - cf['median_LAB_B']).abs()

In [64]:
# Sort by difference and drop duplicates keeping the one with the smallest difference
cfm = cf.sort_values(by='diff').drop_duplicates('catloc', keep='first')

In [65]:
# Optionally, drop the columns used for calculation if they are no longer needed
cfm = cfm.drop(columns=['median_LAB_B', 'diff', 'mtrial', 'mmode', 'catloc'])

In [66]:
cfm = cfm.sort_values(['catalog','mloc'])

In [67]:
cfm = cfm.reset_index(drop=True)

In [68]:
cfm = cfm.rename(columns={"manbran":"manbran_c"})

In [69]:
for dminhex in cfm.rgbhex.loc[cfm.mloc=='dmin']:
    try:
        Image.new('RGB',(10,10),dminhex)
    except:
        print(dminhex)

#107feea
#108ffe9


#### clamp out-of-gamut colors

In [70]:
from colormath.color_objects import LabColor, sRGBColor
from colormath.color_conversions import convert_color

In [71]:
def clamp_hex(df,i):
    l = df.LAB_L.loc[i]
    a = df.LAB_A.loc[i]
    b = df.LAB_B.loc[i]
    
    lab = LabColor(l, a, b)
    rgb = convert_color(lab, sRGBColor)
    rgbhex = sRGBColor(rgb.clamped_rgb_r,rgb.clamped_rgb_g,rgb.clamped_rgb_b).get_rgb_hex()
    
    return rgbhex

In [72]:
idx = cfm.loc[cfm.rgbhex=='#107feea'].index
cfm.rgbhex.loc[idx] = clamp_hex(cfm,idx)

In [73]:
idx = cfm.loc[cfm.rgbhex=='#108ffe9'].index
cfm.rgbhex.loc[idx] = clamp_hex(cfm,idx)

In [74]:
# compose(*list(cfm.rgbhex.loc[cfm.mloc=='dmin'].apply(lambda x: Image.new('RGB',(10,10),x))),thumb=10)
# compose(*list(cfm.rgbhex.loc[cfm.mloc=='dmax'].apply(lambda x: Image.new('RGB',(10,10),x))),thumb=10)

In [75]:
del cfm['fname']
del cfm['LAB_L']
del cfm['LAB_A']

In [76]:
catalogs = []
manbran_cs = []
LAB_Bs_dmin = []
LAB_Bs_dmax = []
rgbhexes_dmin = []
rgbhexes_dmax = []

for catalog in cfm.catalog.unique():
    catalogs.append(catalog)
    
    tmp = cfm.loc[cfm.catalog==catalog]
    assert len(tmp) < 3
    
    assert len(set(tmp.manbran_c))==1
    manbran_cs.append(tmp.manbran_c.iloc[0])
    
    dmin = tmp.loc[tmp.mloc=='dmin']
    if len(dmin) > 0:
        assert len(dmin) == 1
        LAB_Bs_dmin.append(dmin.LAB_B.iloc[0])
        rgbhexes_dmin.append(dmin.rgbhex.iloc[0])
    else:
        LAB_Bs_dmin.append(None)
        rgbhexes_dmin.append(None)
        
    dmax = tmp.loc[tmp.mloc=='dmax']
    if len(dmax) > 0:
        assert len(dmax) == 1
        LAB_Bs_dmax.append(dmax.LAB_B.iloc[0])
        rgbhexes_dmax.append(dmax.rgbhex.iloc[0])
    else:
        LAB_Bs_dmax.append(None)
        rgbhexes_dmax.append(None)

In [77]:
cfmwide = pd.DataFrame({"catalog":catalogs,
                        #"manbran_c":manbran_cs,
                        "dmin":LAB_Bs_dmin,
                        "dmax":LAB_Bs_dmax,
                        "dminHex":rgbhexes_dmin,
                        "dmaxHex":rgbhexes_dmax
                       })

#### corrections

In [78]:
cfmwide = cfmwide.loc[cfmwide.catalog!='214']

In [79]:
idx = cfmwide.loc[cfmwide.catalog=='249'].index
cfmwide = cfmwide.drop(idx)
cfmwide = cfmwide.reset_index(drop=True)
cfmwide.catalog.loc[cfmwide.catalog=='294'] = '249'

In [80]:
cfmwide.catalog.loc[cfmwide.catalog=='5515y'] = '5515z'
cfmwide.catalog.loc[cfmwide.catalog=='5515x'] = '5515y'
cfmwide.catalog.loc[cfmwide.catalog=='5516y'] = '5516z'
cfmwide.catalog.loc[cfmwide.catalog=='5516x'] = '5516y'

In [81]:
sf = pd.read_pickle("/Users/damoncrockett/Desktop/uv/color_measurements_for_uv_analysis.pkl")

In [82]:
sf = sf[['catalog','auc']]

In [83]:
cfmwide = cfmwide.join(sf.set_index('catalog'),'catalog')

In [84]:
cfmwide = cfmwide.loc[~cfmwide.catalog.isin(omits)]

In [85]:
cfmwide.to_pickle('c.pkl')

# Texture image and color reading

In [86]:
cx = cfmwide.join(xfm.set_index('catalog'),'catalog')

In [87]:
cx = cx.loc[(cx.dminHex.notnull())&(cx.roughness.notnull())]

In [88]:
def draw_panel(df,i):
    dminHex = df.dminHex.loc[i]
    pbpath = df.pbpath_new.loc[i]
    canvas = Image.new('RGB',(128,128),dminHex)
    im = Image.open(pbpath)
    im.thumbnail((64,64),Image.Resampling.LANCZOS)
    canvas.paste(im,(32,32))
    
    return canvas

In [89]:
# cx['gpath'] = [draw_panel(cx,i) for i in cx.index]

In [90]:
from ivpy.extract import extract

In [91]:
attach(cx,'pbpath_new')

In [92]:
#cx['brightness'] = extract('brightness')

In [93]:
#montage(pathcol=cx.gpath,xcol=cx.brightness,thumb=128,shape='rect').save('/Users/damoncrockett/Desktop/cx.jpg')

In [94]:
# attach(cx,'pbpath_new')
# show(xcol='brightness',ascending=True,xdomain=(0,0.3),idx=True,thumb=120,bg='grey',
#      notecol=cx.roughness.apply(lambda x: round(x,3)))

In [95]:
# attach(cx,'pbpath_new')
# show(xcol='roughness',xdomain=(0.189,0.6),idx=True,thumb=120,bg='grey',
#      notecol=cx.roughness.apply(lambda x: round(x,3)))

# Join

In [96]:
tmp = db.join(cfmwide.set_index('catalog'),'catalog')
tmp = tmp.join(xfm.set_index('catalog'),'catalog')
tmp = tmp.join(tfm.set_index('catalog'),'catalog')
tmp = tmp.join(gfm.set_index('catalog'),'catalog')

In [97]:
tmp.to_pickle('/Users/damoncrockett/lmlproc/dbdf.pkl')